In [20]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [21]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lebu,CL,-37.6167,-73.6500,50.22,85,96,7.40,overcast clouds
1,1,Vao,NC,-22.6667,167.4833,76.24,84,97,11.79,light rain
2,2,Kaitangata,NZ,-46.2817,169.8464,46.11,64,12,2.64,few clouds
3,3,Vanavara,RU,60.3400,102.2797,73.96,59,80,8.25,light rain
4,4,Toora-Khem,RU,52.4675,96.1100,66.51,37,100,4.68,overcast clouds


In [22]:
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Get the maximum temperature.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Vanavara,RU,60.3400,102.2797,73.96,59,80,8.25,light rain
17,17,Komsomolskiy,UZ,40.4272,71.7189,72.32,53,99,2.30,overcast clouds
23,23,Rikitea,PF,-23.1203,-134.9692,72.59,64,60,2.46,broken clouds
37,37,Puerto Ayora,EC,-0.7393,-90.3518,71.55,92,67,8.01,broken clouds
69,69,Sao Filipe,CV,14.8961,-24.4956,73.72,86,58,6.29,broken clouds
95,95,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,74.07,88,100,8.77,overcast clouds
102,102,Katsuura,JP,35.1333,140.3000,74.77,85,100,15.26,light rain
121,121,Ndele,CF,8.4109,20.6473,70.34,92,100,3.38,overcast clouds
122,122,Touros,BR,-5.1989,-35.4608,72.23,84,24,9.37,few clouds
126,126,Humaita,BR,-7.5061,-63.0208,71.47,98,17,1.59,few clouds


In [33]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")


In [43]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_clean_hotel_df = preferred_cities_df


In [50]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vanavara,RU,73.96,light rain,60.3400,102.2797,
17,Komsomolskiy,UZ,72.32,overcast clouds,40.4272,71.7189,
23,Rikitea,PF,72.59,broken clouds,-23.1203,-134.9692,
37,Puerto Ayora,EC,71.55,broken clouds,-0.7393,-90.3518,
69,Sao Filipe,CV,73.72,broken clouds,14.8961,-24.4956,
95,Sao Jose Da Coroa Grande,BR,74.07,overcast clouds,-8.8978,-35.1478,
102,Katsuura,JP,74.77,light rain,35.1333,140.3000,
121,Ndele,CF,70.34,overcast clouds,8.4109,20.6473,
122,Touros,BR,72.23,few clouds,-5.1989,-35.4608,
126,Humaita,BR,71.47,few clouds,-7.5061,-63.0208,


In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame.
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")
        

Hotel not found... skipping.3
Hotel not found... skipping.121
Hotel not found... skipping.293
Hotel not found... skipping.321
Hotel not found... skipping.400
Hotel not found... skipping.442
Hotel not found... skipping.586
Hotel not found... skipping.681


In [52]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vanavara,RU,73.96,light rain,60.3400,102.2797,
17,Komsomolskiy,UZ,72.32,overcast clouds,40.4272,71.7189,Ikathouse
23,Rikitea,PF,72.59,broken clouds,-23.1203,-134.9692,People ThankYou
37,Puerto Ayora,EC,71.55,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
69,Sao Filipe,CV,73.72,broken clouds,14.8961,-24.4956,Hotel Xaguate
...,...,...,...,...,...,...,...
654,Freeport,US,70.47,overcast clouds,40.6576,-73.5832,The Freeport Inn and Marina
667,Broome,US,71.85,mist,42.2506,-75.8330,Chenango Valley State Park
681,San Lazaro,BZ,73.58,overcast clouds,18.0387,-88.6622,
685,Biu,NG,70.14,overcast clouds,10.6129,12.1946,Emir's Palace


In [53]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))